In [1]:
import pandas as pd
import numpy as np

In [4]:
import os
print("Working directory:", os.getcwd())
path = os.path.abspath('../data/cleaned_data/infrastructure/BMMS_overview.xlsx')
print("Absolute path being used:", path)
print("Exists?", os.path.exists(path))

# # Force the working dir to the 'model' folder
#os.chdir(r"C:\Users\domho\OneDrive\Documents\GitHub\AdvancedSimulation\EPA133a-G04-A2\model")

Working directory: C:\Program Files\JetBrains\PyCharm 2024.3\bin
Absolute path being used: C:\Program Files\JetBrains\PyCharm 2024.3\data\cleaned_data\infrastructure\BMMS_overview.xlsx
Exists? False


In [5]:
bridges = pd.read_excel('../data/cleaned_data/infrastructure/BMMS_overview.xlsx')
roads = pd.read_csv('../data/cleaned_data/infrastructure/_roads3.csv')
roads = roads[roads['road'] == 'N1']


In [6]:
# Filter only bridges related to road N1
bridges_n1 = bridges[bridges['road'] == 'N1']

# Rename columns to align with roads DataFrame (type -> model_type)
bridges_n1 = bridges_n1.rename(columns={"type": "model_type"})
bridges_n1 = bridges_n1.drop(columns=['length'])

# Add necessary default values for additional columns
bridges_n1['length'] = bridges_n1["chainage"].diff().abs().fillna(0)    # Default length for all bridges
bridges_n1['model_type'] = "bridge"  # Standardize as "bridge"

# Remove unnecessary columns from bridges to match roads (if any)
if 'lrp' in bridges_n1.columns:
    bridges_n1 = bridges_n1.drop(columns=['lrp'])
if 'gap' in bridges_n1.columns:
    bridges_n1 = bridges_n1.drop(columns=['gap'])

# Keeping only the relevant and previously mentioned columns
bridges_n1 = bridges_n1[['road','name','chainage','condition', 'model_type', 'length','lat','lon']]
# Append bridges to the roads DataFrame

# Save the updated DataFrame back to the CSV file
# roads.to_csv("../data/cleaned_data/infrastructure/updated_roads.csv", index=False)

# Display the first few rows for verification
bridges_n1.head(20)

,road,name,chainage,condition,model_type,length,lat,lon
0,N1,.,1.800,A,bridge,0.000,23.698739,90.458861
1,N1,.,4.925,A,bridge,3.125,23.694664,90.487775
2,N1,Kanch pur Bridge.,8.976,A,bridge,4.051,23.705060,90.523214
3,N1,NOYAPARA CULVERT,10.880,A,bridge,1.904,23.694391,90.537574
4,N1,ADUPUR CULVERT,10.897,A,bridge,0.017,23.694302,90.537707
5,N1,NAYABARI KASPUR BOX CULVERT,11.296,A,bridge,0.399,23.692360,90.540918
6,N1,KHAS PARA BOX CULVERT,12.239,A,bridge,0.943,23.688412,90.548559
7,N1,DAWAN BAG BOX CULVERT,12.253,A,bridge,0.014,23.688320,90.548650
8,N1,Madanpur Bridge.(L),12.660,A,bridge,0.407,23.685583,90.551208
9,N1,MADAN PUR (R),12.660,A,bridge,0.000,23.685583,90.551208


In [7]:
bridges["type"] = bridges["type"].astype(str).str.lower()
b = bridges["type"].unique()
b = b.tolist()
b.append("culvert")
b.append("bridge")
b = np.array(b)
print(b)


['box culvert' 'pc girder bridge' 'pc box' 'rcc girder bridge'
 'slab culvert' 'steel beam & rcc slab' 'arch masonry' 'rcc bridge'
 'truss with rcc slab' 'baily with steel deck' 'truss with steel deck'
 'baily with timber deck' 'truss with timber deck' 'pipe culvert'
 'culvert' 'bridge']


In [8]:
# Rename and reformat the columns
roads["length"] = roads["chainage"].diff().abs().fillna(0)
roads["type"] = roads["type"].astype(str).str.lower()

# Update 'type' column values to standardize naming
roads.loc[roads["type"].isin(b), "type"] = "bridge"
roads.loc[roads["type"] != "bridge", "type"] = "link"

# Correct assignment for model_type using .loc for single-element assignment
roads.loc[0, "type"] = "source"  # Assign 'source' to the first row
roads.loc[len(roads.index) - 1, "type"] = "sink"  # Assign 'sink' to the last row

# Rename column 'type' to 'model_type'
roads = roads.rename(columns={
    "type": "model_type",
})

# Drop unnecessary columns
roads = roads.drop('lrp', axis=1)
roads = roads.drop('gap', axis=1)

# Display the first 40 rows for verification
roads.head()

,road,chainage,lat,lon,model_type,name,length
0,N1,0.000,23.706028,90.443333,source,Start of Road after Jatrabari Flyover infront...,0.000
1,N1,0.814,23.702917,90.450417,bridge,Box Culvert,0.814
2,N1,0.822,23.702778,90.450472,link,Intersection with Z1101,0.008
3,N1,1.000,23.702139,90.451972,link,Km post missing,0.178
4,N1,2.000,23.697889,90.460583,link,Km post missing,1.000


In [9]:
# Step 3: Append it to the roads DataFrame
roads = pd.concat([roads, bridges_n1], ignore_index=True)

# Step 4: Sort the updated DataFrame by chainage
roads = roads.sort_values(by='chainage').reset_index(drop=True)

roads["id"] = roads.index
roads["id"] = (roads["id"] + 1_000_000).astype(int)

In [11]:
roads.loc[(roads['condition'].isna()) & (roads['model_type'] == 'bridge'), 'condition'] = 'A'

# 1. Identify duplicate chainages
duplicate_chainages = roads[roads.duplicated(subset="chainage", keep=False)]

# 2. Mark bridges with "(L)" in the name
roads["has_L"] = roads["name"].str.contains(r"\(L\)", case=False, na=False)

# 3. Sort duplicates so that bridges with "(L)" come first, then by other criteria if needed
df_sorted = roads.sort_values(by=["chainage", "has_L"], ascending=[True, False])

# 4. Drop duplicates, keeping only the first occurrence (which is prioritized due to sorting)
df_unique = df_sorted.drop_duplicates(subset="chainage", keep="first")

# 5. Drop the helper column "has_L" to clean up the DataFrame
df_unique = df_unique.drop(columns=["has_L"])
roads = df_unique

#make sure the length is numeric
roads["length"] = pd.to_numeric(roads["length"], errors="coerce")

roads.to_csv("../data/cleaned_data/infrastructure/roads_for_model.csv", index=False)
roads.head(20)

,road,chainage,lat,lon,model_type,name,length,condition,id
0,N1,0.000,23.706028,90.443333,source,Start of Road after Jatrabari Flyover infront...,0.000,NaN,1000000
1,N1,0.814,23.702917,90.450417,bridge,Box Culvert,0.814,A,1000001
2,N1,0.822,23.702778,90.450472,link,Intersection with Z1101,0.008,NaN,1000002
3,N1,1.000,23.702139,90.451972,link,Km post missing,0.178,NaN,1000003
4,N1,1.800,23.698739,90.458861,bridge,.,0.000,A,1000004
5,N1,2.000,23.697889,90.460583,link,Km post missing,1.000,NaN,1000005
6,N1,2.130,23.697361,90.461667,bridge,Box culvert,0.130,A,1000006
7,N1,3.000,23.693833,90.469138,link,Km post missing,0.870,NaN,1000007
8,N1,4.000,23.693611,90.478777,link,Km post missing,1.000,NaN,1000008
9,N1,4.175,23.693805,90.480527,link,Road to Narayanganj(R111),0.175,NaN,1000009
